In [155]:
import os
import math
import tarfile
from six.moves import urllib
import hashlib
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TreebankWordTokenizer
from bs4 import BeautifulSoup
import xml.etree.ElementTree
import mailbox   
import lxml.html
import lxml.html.clean
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.stem import PorterStemmer
import email
from sklearn.pipeline import Pipeline
# Import the email modules we'll need
from email.parser import Parser
import mailparser as mp
from email.mime.text import MIMEText
import nltk
from nltk import pos_tag
import re
from collections import Counter



# path = spam_path
# i = 0
# for filename in os.listdir(path):
#     print('orig', filename)
#     os.rename(os.path.join(path,filename), os.path.join(path,'mail_'+str(i)))
#     print('changed', filename)
#     i = i +1
# print(i)




class preparing_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, x, y=None):
        return self
    def transform(self, x, y=None):
        #print('type of x is' , type(x))
        #x = x.lower()
        m_body, m_headers, sub = dividing_mail(x)
        if m_body != '':
            m_body = m_body.lower()
            #m_body = m_body.split('.').join("");
            urls = counting_urls(m_body)
            m_body = cleaning_htmlent(m_body)
            
            name_sender, dom_sender = x.from_[0][0], x.from_[0][1].split('@')[-1]
        
            sub_list = keep_only_important_words(sub)
            body_list = keep_only_important_words(m_body)
            sub_list = removing_alphas(sub_list)
            body_list = removing_alphas(body_list)
            
            ps = PorterStemmer()
            s_c =   [ps.stem(w) for w in sub_list]
            b_c =   [ps.stem(w) for w in body_list]
            b_c.extend([urls, name_sender, dom_sender])
        else:
            s_c =[]
            b_c = []
        return s_c, b_c
        



            
def keep_only_important_words(data_txt):
        m_words =[]
        for word, pos in nltk.pos_tag(TreebankWordTokenizer().tokenize(data_txt)):
            #if pos not in ['IN', '.', 'CD', 'DT', 'RB', 'VBP', 'TO', 'PRP', 'C']: VB and VBP removed
            if pos in ['NN',  'JJ', 'VBN'] :
                m_words.append(word)
            else:
                pass
        return(m_words)

def cleaning_htmlent(text):
    if text != '':
        doc = lxml.html.fromstring(text)
        cleaner = lxml.html.clean.Cleaner(style=True)
        doc = cleaner.clean_html(doc)
        content = doc.text_content()
        return content
    else:
        return []

def counting_urls(text):
#     soup = BeautifulSoup(text, 'html.parser')
#     num = soup.find_all('a')
#     print('number is ', (num))

    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    num_urls = len(urls)
    return num_urls
  
def dividing_mail(mail):
        m_body = mail.body
        m_headers = mail.headers
        
        sub = mail.subject
        return m_body, m_headers, sub

sub_list = []
body_list = []



def removing_alphas(doc):
    dellist = []
    for i in range(len(doc)):
        if not doc[i].isalpha():
            dellist.append(i)
       
    for index in sorted(dellist, reverse=True):
        del doc[index]
    return doc
    

#Giving filepaths for the data
dat_path = "datasets/spamham"
spam_path = dat_path + '/easy_spam'
ham_path = dat_path + '/easy_ham'
len_spam = len(next(os.walk(spam_path))[2])
len_ham = len(next(os.walk(ham_path))[2])
ar_dict = [(spam_path, len_spam)] #, {ham_path, len_ham}] 501

y_train = [0] * math.ceil(0.8 * len_spam) + [1] * math.ceil(0.8 * len_spam)
n_list = []
    
for sp, num in ar_dict:
    tr_n = math.ceil(0.8 * int(num))
    
    for file_num in range(tr_n):
        #Reading an email and getting the content
        data_mail = spam_path + '/mail_' + str(file_num)
        #print(data_mail)
        
        
        mail = mp.parse_from_file(data_mail)
        obj = preparing_data()
        s, b = obj.fit_transform(mail)
        n_list.append(b[-3:])
        sub_list.append(s)
        body_list.append(b)

        #print('length of sub is ', len(sub_list))
        #print('length of body is ', len(body_list))
    
        #words_mails.append(l)
        #print(len(words_mails))

b_fv = extracting_common(body_list, 35)
s_fv = extracting_common(sub_list, 3)
print(len(b_fv))
featurevector = b_fv + s_fv + n_list
#print(featurevector)

def extracting_common(words_list, to_choose):
    flatten_list = [a for t in words_list for a in t]
    out = Counter(flatten_list)
    #out = out.most_common(to_choose)
    out = [(k, v) for k, v in out.items() if v >= to_choose]
    #l_words = [tup[0] for tup in out]
    return out


242


In [151]:
flatten_list = [a for t in sub_list for a in t]
out = Counter(flatten_list)
out

d = [(k, v) for k, v in out.items() if v >= 2]
#out = out.most_common(to_choose)
#out
d

[('year', 10),
 ('Re', 15),
 ('Fw', 3),
 ('invest', 6),
 ('adv', 14),
 ('life', 3),
 ('insur', 3),
 ('avail', 4),
 ('mood', 2),
 ('onlin', 3),
 ('prescript', 2),
 ('qeeb', 2),
 ('request', 2),
 ('competit', 3),
 ('FW', 4),
 ('kind', 3),
 ('money', 9),
 ('maker', 3),
 ('free', 23),
 ('market', 8),
 ('exquisit', 2),
 ('softwar', 3),
 ('first', 2),
 ('fwd', 5),
 ('ilug', 2),
 ('poker', 2),
 ('other', 3),
 ('trial', 3),
 ('しじみともものコラボレーション', 3),
 ('shipment', 2),
 ('statu', 2),
 ('plan', 2),
 ('futur', 2),
 ('adult', 3),
 ('systemwork', 3),
 ('RE', 2),
 ('guarante', 4),
 ('mortgag', 8),
 ('rate', 3),
 ('tobacco', 2),
 ('home', 2),
 ('own', 2),
 ('work', 2),
 ('quot', 2),
 ('busi', 5),
 ('pleas', 2),
 ('domain', 3),
 ('extend', 5),
 ('low', 11),
 ('net', 2),
 ('loss', 2),
 ('attn', 6),
 ('few', 2),
 ('minut', 3),
 ('zzzz', 4),
 ('last', 3),
 ('ebay', 6),
 ('big', 2),
 ('cabl', 3),
 ('chines', 2),
 ('invit', 2),
 ('new', 3),
 ('interest', 3),
 ('cost', 2),
 ('confer', 2),
 ('shop', 2),
 ('dis

In [131]:
flatten_list = [a for t in body_list for a in t]
print(flatten_list)

['import', 'inform', 'new', 'domain', 'avail', 'gener', 'public', 'discount', 'new', 'domain', 'origin', 'brand', 'new', 'domain', 'approv', 'icann', 'same', 'origin', 'domain', 'benefit', 'domain', 'attract', 'domain', 'name', 'same', 'visit', 'http', 'today', 'regist', 'domain', 'name', 'today', 'http', 'registr', 'full', 'access', 'control', 'panel', 'domain', 'name', 'domain', 'administr', 'afford', 'email', 'address', 'further', 'promot', 'compani', 'click', 'http', 3, '', 'emailaccount.com', 'dear', 'free', 'member', 'gift', 'free', 'membership', 'cash', 'wave', 'membership', 'lock', 'cash', 'incom', 'free', 'interest', 'posit', 'chanc', 'link', 'mailto', 'workinathom', 'email', 'posit', 'cash', 'wave', 'gener', 'administr', 'exclud', 'futur', 'mailto', 0, 'Mandy', 'yahoo.co.uk', 'wholesal', 'free', 'ship', 'pay', 'state', 'mail', 'swiss', 'bond', 'person', 'use', 'age', 'verifi', 'credit', 'http', 'aol', 'remov', 'futur', 'repli', 'email', 'remov', 'subject', 'line', 2, 'Discoun

In [107]:
m_words =[]
data_txt = 'i am and happy your'
for word, pos in nltk.pos_tag(TreebankWordTokenizer().tokenize(data_txt)):
    if pos not in ['IN', '.', 'CD', 'DT', 'RB', 'VBP']:
        print (word, pos)

i NN
and CC
happy JJ
your PRP$


In [133]:
urls = re.findall('anau', 'anadu is an  adf  eargf er erg reg ret')
num_urls = len(urls)
num_urls

0